# Лекция 3: Деревья решений и ансамблевые методы

### Деревья решений

#### Принцип работы

1. **Корневой узел**: Всю выборку $D = \{\mathbf{x}, \mathbf{y}\}$ рассматривают как набор в корневом узле.
2. **Разбиение**: На каждом шаге выбирается признак и порог, по которому выборка разделяется на два (или более) подмножеств $D_1$ и $D_2$. Критерий выбора разбиения основан на уменьшении некоторой величины, показывающей качество (чистоту) разбиения.
3. **Остановка**: Процесс рекурсивного деления продолжается до достижения определённых условий (максимальная глубина, минимальное число объектов в узле, достижение чистого узла и т.д.).

#### Метрики для разбиения

##### Энтропия и информационный выигрыш

Энтропия – это мера неопределённости или «хаотичности» распределения. Если в узле дерева классы распределены поровну, энтропия максимальна (мы практически не можем предсказать класс случайно выбранного объекта). Если же все объекты одного класса, энтропия равна нулю – неопределённость отсутствует. При выборе признака для разбиения мы ищем тот, который даёт наибольшее снижение энтропии (то есть увеличивает чистоту разбиения).

Предположим, что в некотором узле дерева у нас есть объекты, принадлежащие K классам, и пусть $p_i$ – доля объектов, принадлежащих классу $i$ (при этом $\sum_{i=1}^{K} p_i = 1$).

Энтропия узла $D$ определяется как:
$$
H = - \sum_{i=1}^{K} p_i \log_2(p_i),
$$
где $p_k$ – доля объектов, принадлежащих классу $k$. 

При этом, если какая-то вероятность равна нулю, принято считать, что вклад $p_i \log_2(p_i) = 0$. 

В случае бинарной классификации (K=2), если обозначить $p = p_1$ и $1-p = p_2$, формула принимает вид:
$$
H = - \left[p \log_2(p) + (1-p) \log_2(1-p)\right]
$$

Например, если $p = 0.5$, то:
$$
H = - \left[0.5\log_2(0.5) + 0.5\log_2(0.5)\right] = - \left[0.5(-1) + 0.5(-1)\right] = 1
$$

При разбиении узла информационный выигрыш вычисляется по формуле:
$$
\Delta H = H(D) - \sum_{v \in \text{Values}(A)} \frac{|D_v|}{|D|} H(D_v),
$$
где $D_v$ – подмножество данных, для которых значение признака $A$ равно $v$.

Например, если исходная неоднородность в родительском узле равна $H_{\text{parent}}$, а после разбиения получаются два дочерних узла с неоднородностями $H_{\text{left}}$ и $H_{\text{right}}$, прирост информации определяется как:
$$
\Delta H = H_{\text{parent}} - \left(\frac{|D_{\text{left}}|}{|D|} H_{\text{left}} + \frac{|D_{\text{right}}|}{|D|} H_{\text{right}}\right),
$$
где $|D_{\text{left}}$ и $|D_{\text{right}}$ – число объектов в левых и правых узлах, а $|D|$ – общее число объектов в родительском узле.

##### Индекс Джини

Индекс Джини – ещё одна мера неоднородности. Он показывает вероятность того, что случайно выбранный объект будет неправильно классифицирован, если назначить ему класс по распределению в узле. Если в узле объекты принадлежат разным классам почти поровну, индекс Джини будет высоким, а если почти все объекты принадлежат к одному классу – низким. При выборе признака для разбиения выбирают тот, который приводит к наименьшему значению индекса Джини в дочерних узлах.

Индекс Джини определяется как:
$$
G = 1 - \sum_{i=1}^{K} p_i^2.
$$

Для бинарного случая получаем:
$$
G = 1 - \left[p^2 + (1-p)^2\right]
$$

Если $p = 0.5$, то:
$$
G = 1 - \left[0.25 + 0.25\right] = 0.5
$$

При выборе разбиения ищут признак, который даёт максимальное уменьшение индекса Джини:
$$
\Delta G = G(D) - \sum_{v \in \text{Values}(A)} \frac{|D_v|}{|D|} G(D_v).
$$

#### Особенности
- **Жадный алгоритм**: На каждом шаге выбирается локально оптимальное разбиение, что не гарантирует глобально оптимального дерева.
- **Переобучение**: Очень глубокие деревья могут переобучаться, поэтому применяются методы отсечения (pruning) для улучшения обобщающей способности модели.
- **Интерпретируемость**: Деревья решений являются интуитивно понятными и наглядно демонстрируют процесс принятия решений.

> Более подробное описание, с алгоритмическими тонкостями и обоснованиями в [учебнике](https://education.yandex.ru/handbook/ml/article/reshayushchiye-derevya)

<img src="../../images/decision_tree.png" alt="Дерево решений" width="500">

Алгоритм построения решающего дерева:
1. **Начальный этап (корневой узел)**:  
   На вход подаётся весь набор данных. Для этого узла вычисляются пропорции классов $p_i$ (например, для бинарной классификации – это $p$ и $1-p$).
2. **Выбор оптимального разбиения**:  
   Для каждого признака и для каждого возможного порогового значения рассчитывается мера неоднородности узла.
   Затем для каждого потенциального разбиения вычисляется прирост информации (или снижение неоднородности).
3. **Разбиение узла**:  
   Выбирается разбиение, при котором прирост информации максимален (или индекс Джини минимален). Данные разделяются на две группы - например, по условию «значение признака $\leq$ порога» и «значение признака $>$ порога».
4. **Рекурсивное повторение**:  
   Применяется тот же алгоритм к каждому из полученных дочерних узлов. Алгоритм повторяет выбор оптимального разбиения для каждого узла, используя те же формулы, пока не выполнится одно из условий остановки:
   - Узел достиг максимальной глубины.
   - Количество объектов в узле меньше заданного минимума.
   - Все объекты в узле принадлежат одному классу.
5. **Окончательное решение (листовые узлы)**:  
   После завершения рекурсии узлы, которые больше не делятся, становятся листовыми узлами.  
   - Для задачи классификации в листе назначается класс, который чаще всего встречается среди объектов.
   - Для задачи регрессии вычисляется среднее значение целевой переменной в узле.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [ ]:
np.random.seed(42)
X, y = make_classification(n_samples=50,
                            n_features=2,
                            n_redundant=0,
                            n_informative=2,
                            n_clusters_per_class=1,
                            flip_y=0.1,
                            class_sep=1.5,
                            random_state=42)

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=42)

In [ ]:
tree_clf = DecisionTreeClassifier(random_state=42)
tree_clf.fit(X_train, y_train)

y_pred = tree_clf.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(acc * 100))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))
print("Report:\n", classification_report(y_test, y_pred))

plt.figure(figsize=(8, 6))
plot_decision_boundary(tree_clf, X, y)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))
plot_tree(tree_clf,
            feature_names=["Feature 1", "Feature 2"],
            filled=True,
            rounded=True,
            fontsize=10)
plt.title("Tree graph")
plt.tight_layout()
plt.show()

### Разложение ошибки на смещение и дисперсию

Одним из центральных понятий в машинном обучении является **разложение ошибки на смещение и дисперсию (bias-variance decomposition)**. Оно позволяет понять, какие составляющие вносят вклад в общую ошибку модели и как можно управлять этим балансом при выборе и настройке моделей.

> Более подробно в [учебнике](https://education.yandex.ru/handbook/ml/article/bias-variance-decomposition)

#### Теория
Пусть у нас имеется функция истинной зависимости $ y = f(x) + \varepsilon $, где:
- $ f(x) $ – истинная функция,
- $\varepsilon$ – случайный шум с дисперсией $\sigma^2$ (неизбежная ошибка, относящаяся к шуму).

Модель, обученная на случайной выборке, даёт предсказание $\hat{f}(x)$, которое само по себе является случайной величиной (так как зависит от конкретной выборки).


Для фиксированного $x$ нас интересует ожидаемая квадратичная ошибка:
$$
\mathbb{E}[(y - \hat{f}(x))^2] = \mathbb{E}[(f(x) + \epsilon - \hat{f}(x))^2].
$$

Раскроем квадрат:
$$
\mathbb{E}\Big[(f(x)-\hat{f}(x))^2\Big] + 2\mathbb{E}\Big[(f(x)-\hat{f}(x))\epsilon\Big] + \mathbb{E}[\epsilon^2].
$$

Поскольку $\epsilon$ имеет нулевое среднее и независим от $\hat{f}(x)$, второй член равен нулю, а $\mathbb{E}[\epsilon^2]=\sigma^2$. 

Теперь для первого члена добавим и вычтем среднее значение предсказания $\mathbb{E}[\hat{f}(x)]$:
$$
f(x)-\hat{f}(x)= \Bigl(f(x)-\mathbb{E}[\hat{f}(x)]\Bigr) + \Bigl(\mathbb{E}[\hat{f}(x)]-\hat{f}(x)\Bigr).
$$

Тогда, раскрыв квадрат, получим:
$$
\mathbb{E}[(f(x)-\hat{f}(x))^2] = \Bigl(f(x)-\mathbb{E}[\hat{f}(x)]\Bigr)^2 + \mathbb{E}\Bigl[(\hat{f}(x)-\mathbb{E}[\hat{f}(x)])^2\Bigr] + 2\,\mathbb{E}\Bigl[\Bigl(f(x)-\mathbb{E}[\hat{f}(x)]\Bigr)\Bigl(\mathbb{E}[\hat{f}(x)]-\hat{f}(x)\Bigr)\Bigr].
$$

При этом последний (поперечный) член равен нулю, поскольку $\mathbb{E}[\hat{f}(x)]- \hat{f}(x)$ имеет нулевое математическое ожидание. Таким образом получаем:
$$
\mathbb{E}\left[(y - \hat{f}(x))^2\right] = \underbrace{\left( \mathbb{E}[\hat{f}(x)] - f(x) \right)^2}_{\text{Bias}^2} + \underbrace{\mathbb{E}\left[\left(\hat{f}(x) - \mathbb{E}[\hat{f}(x)]\right)^2\right]}_{\text{Variance}} + \sigma^2.
$$

- **Смещение (Bias)**: Показывает, насколько среднее предсказание модели отклоняется от истинной функции $ f(x) $. Высокое смещение может возникать, если модель слишком проста и не способна уловить сложность зависимости (так называемое недообучение).

- **Дисперсия (Variance)**: Характеризует изменчивость предсказаний модели при изменении обучающей выборки. Высокая дисперсия свойственна сложным моделям, которые сильно адаптируются к обучающим данным (что может привести к переобучению).

Баланс между bias и variance определяет качество обобщения модели. Идеальной моделью является такая, которая имеет как низкое смещение, так и низкую дисперсию, однако в реальных задачах часто наблюдается компромисс между ними.

#### Сильные и слабые модели
В контексте ансамблирования модели часто делят на **слабые** и **сильные**:

- **Слабые модели** – это модели, которые лишь немного лучше случайного угадывания. В задачах классификации такие алгоритмы называют «слабым обучением». Пример: *решающее дерево с единичной глубиной* (decision stump) или в целом неглубокие деревья. Они, как правило, имеют высокое смещение (не могут уловить сложные зависимости), но обладают низкой дисперсией.

- **Сильные модели** – это более сложные модели с хорошей способностью описывать данные (низкое смещение), но зачастую они обладают высокой дисперсией. Пример: полные, глубоко растущие деревья решений без отсечения. В ансамблировании, например, случайный лес использует именно такие сильные модели, и за счёт бутстреп-агрегации (bagging) достигается значительное снижение дисперсии.

Различные методы ансамблирования используют именно эти особенности:

- **Бэггинг (Bagging)**: Обычно применяется для сильных моделей с высоким разбросом (variance). Например, полные деревья решений могут сильно колебаться от разбиения к разбиению, и их усреднение снижает дисперсию предсказания.

- **Бустинг (Boosting)**: Часто использует слабые модели с ограниченной глубиной (низкая дисперсия, высокое смещение), внося последовательные коррективы в модель. Каждая последующая слабая модель обучается на исправлении ошибок предыдущих, что позволяет снижать смещение итогового ансамбля.


<img src="../../images/darts.png" alt="Визуализация bias и variance" width="500">

<img src="../../images/bias_vs_variance.png" alt="Сложность модели и переобучение" width="500">

In [29]:
!pip install mlxtend seaborn -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from mlxtend.evaluate import bias_variance_decomp

In [ ]:
np.random.seed(42)
X, y = make_moons(n_samples=500, noise=0.3, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=1)

In [ ]:
clf = DecisionTreeClassifier(random_state=1,
                             max_depth = 100)
avg_error, avg_bias, avg_var = bias_variance_decomp(clf, 
                                                    X_train, y_train, 
                                                    X_test, y_test,
                                                    loss='0-1_loss', 
                                                    num_rounds=200, 
                                                    random_seed=1)

print("Average Classification Error: {:.3f}".format(avg_error))
print("Average Bias: {:.3f}".format(avg_bias))
print("Average Variance: {:.3f}".format(avg_var))

labels = ['Error', 'Bias', 'Variance']
values = [avg_error, avg_bias, avg_var]

plt.figure(figsize=(8, 6))
colors = sns.color_palette("Set2", n_colors=len(labels))
bars = plt.bar(labels, values, color=colors)
plt.ylabel("Value")
plt.title("Bias-Variance for Decision Tree Classifier")
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, yval, 
             f"{yval:.3f}", ha='center', va='bottom')
plt.tight_layout()
plt.show()

### Ансамблевые методы

Ансамблевые методы объединяют несколько моделей для получения более точного и стабильного результата. Они используются для уменьшения дисперсии, смещения или общей ошибки модели.

> [Учебник](https://education.yandex.ru/handbook/ml/article/ansambli-v-mashinnom-obuchenii)

<img src="../../images/ensemble_methods.png" alt="Ансамблевые методы" width="600">

<img src="../../images/ensemble_models.png" alt="Ансамблевые модели" width="600">

In [ ]:
import numpy as np

In [ ]:
def true_function(x):
    """
    Истинная функция: если sin(x) > 0, то класс 1, иначе 0
    """
    return (np.sin(x) > 0).astype(int)

In [ ]:
def run_experiments(model, n_train, noise_std, x_grid, n_experiments):
    """
    Запускает серию экспериментов для оценки предсказаний модели.

    Аргументы:
        model: экземпляр модели (например, DecisionTreeRegressor или RandomForestRegressor)
        n_train (int): число обучающих образцов в каждом эксперименте.
        noise_std (float): стандартное отклонение шума.
        x_grid (np.ndarray): массив точек (форма (n_points, 1)) для вычисления предсказаний.
        n_experiments (int): количество экспериментов.

    Возвращает:
        predictions (np.ndarray): массив предсказаний формы (n_experiments, n_points).
    """
    predictions = []
    for _ in range(n_experiments):
        X_train = np.random.uniform(0, 2*np.pi, n_train).reshape(-1, 1)
        noise = np.random.normal(0, noise_std, n_train)
        y_train = true_function(X_train.ravel())

        model.fit(X_train, y_train)
        pred = model.predict(x_grid)
        predictions.append(pred)
    return np.array(predictions)

In [ ]:
def bias_variance_decomposition(predictions, y_true):
    """
    Вычисляет bias² и variance для набора предсказаний по координатам.

    Аргументы:
        predictions (np.ndarray): матрица предсказаний формы (n_experiments, n_points).
        y_true (np.ndarray): истинные значения функции для точек сетки (n_points,).

    Возвращает:
        global_bias (float): усреднённое значение bias² по всем точкам сетки.
        global_variance (float): усреднённое значение дисперсии предсказаний.
    """
    mean_pred = np.mean(predictions, axis=0)
    bias_sq = (y_true - mean_pred)**2
    variance = np.var(predictions, axis=0)
    global_bias = np.mean(bias_sq)
    global_variance = np.mean(variance)
    return global_bias, global_variance

In [ ]:
def display_results(model_names, bias_values, variance_values):
    """
    Выводит результаты (bias² и variance для каждой модели) и строит группированный столбчатый график.

    Аргументы:
        model_names (list[str]): список имен моделей.
        bias_values (list[float]): список значений bias² для каждой модели.
        variance_values (list[float]): список значений variance для каждой модели.
    """
    for name, bias, var in zip(model_names, bias_values, variance_values):
        print(f"{name}:")
        print(f"  Bias²: {bias:.3f}, Variance: {var:.3f}\n")

    _, ax = plt.subplots(figsize=(6, 4))
    width = 0.35
    indices = np.arange(len(model_names))

    palette = sns.color_palette("Set2", n_colors=2)
    bars_bias = ax.bar(indices - width / 2, bias_values, width, 
                       label="Bias²", color=palette[0])
    bars_variance = ax.bar(indices + width / 2, variance_values, width, 
                           label="Variance", color=palette[1])

    ax.set_xlabel("Модель")
    ax.set_ylabel("Значение")
    ax.set_xticks(indices)
    ax.set_xticklabels(model_names)
    ax.legend()

    for bar in bars_bias + bars_variance:
        height = bar.get_height()
        ax.annotate(f"{height:.3f}",
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha="center", va="bottom")

    plt.tight_layout()
    plt.show()

#### Bagging

**Bootstrap Aggregating (Bagging)** заключается в обучении множества моделей на различных бутстреп-выборках из исходного набора данных. Итоговый прогноз вычисляется как агрегированное (например, голосование для классификации) предсказание отдельных 
моделей.

<div class="alert alert-danger">
Бутстреп выборка имеет такой же размер, что и исходная (генерация с повторениями).
</div> 

Пусть $ D $ – исходная выборка из $ N $ объектов. Из неё генерируются $ M $ бутстреп-выборок $ D^{(1)}, D^{(2)}, \ldots, D^{(M)} $. Для каждой выборки $ D^{(m)} $ обучается модель $ f^{(m)}(x) $. Тогда итоговая модель имеет вид:
  $$
  f_{\text{bag}}(x) = \frac{1}{M}\sum_{m=1}^{M} f^{(m)}(x).
  $$
  Для задач классификации итоговое решение часто определяется по принципу большинства голосов.

<div class="alert alert-info">
Особенность такого подхода в снижении дисперсии модели, стабилизации предсказаний и уменьшения риска переобучения.
</div> 

**Пример:**
    Random Forest – ансамбль деревьев решений, где каждая модель обучается на бутстреп-выборке, а при выборе разбиений учитывается случайное подмножество признаков.

- Каждое дерево обучается на случайной бутстреп-выборке исходных данных.
- При построении каждого дерева на каждом узле выбирается случайное подмножество признаков для разбиения, что снижает корреляцию между деревьями.

Для $ M $ деревьев решений, где $ f^{(m)}(x) $ – предсказание $ m $-го дерева, итоговое предсказание определяется как:
$$
f_{\text{RF}}(x) = \frac{1}{M}\sum_{m=1}^{M} f^{(m)}(x),
$$
при этом для задач классификации применяется голосование, а для регрессии – усреднение.

<img src="../../images/bagging_classifier.png" alt="Bagging" width="600">

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
np.random.seed(42)
n_train = 20
noise_std = 0.3
n_experiments = 50
x_grid = np.linspace(0, 2*np.pi, 100).reshape(-1, 1)
y_true = true_function(x_grid.ravel())

tree_model = DecisionTreeClassifier(random_state=42, max_depth=5)
tree_predictions = run_experiments(tree_model, n_train, 
                                   noise_std, x_grid, n_experiments)
tree_bias, tree_variance = bias_variance_decomposition(tree_predictions, y_true)

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_predictions = run_experiments(rf_model, n_train, noise_std, x_grid, n_experiments)
rf_bias, rf_variance = bias_variance_decomposition(rf_predictions, y_true)

model_names = ["Decision Tree", "Bagging"]
bias_values = [tree_bias, rf_bias]
variance_values = [tree_variance, rf_variance]

display_results(model_names, bias_values, variance_values)

#### Boosting

Бустинг (Boosting) – это семейство ансамблевых методов, в которых базовые модели (`weak learners`) обучаются **последовательно**. Основная идея в том, что каждая следующая модель стремится скорректировать ошибки предыдущих. В результате достигается высокое качество за счёт комбинирования многих слабых моделей в один сильный ансамбль.

##### AdaBoost

**AdaBoost** – один из первых бустинговых алгоритмов для классификации. Основная идея – обучать последовательность слабых классификаторов $h_m$, каждый из которых уделяет больше внимания объектам, неправильно классифицированным предыдущими моделями.

1. **Инициализация весов объектов**

   $$
   w_i^{(1)} = \frac{1}{N}, \quad i=1,\dots,N.
   $$

2. **Обучение $m$-го слабого классификатора**
   $h_m: \mathcal X \to \{ -1, +1\}$ обучается на выборке с весами $w_i^{(m)}$.

3. **Вычисление ошибки модели**

   $$
   \epsilon_m = \sum_{i=1}^{N} w_i^{(m)} \; \mathbb{I}\bigl(y_i \neq h_m(x_i)\bigr),
   $$

   где $y_i \in \{-1, +1\}$.

4. **Вес модели в ансамбле**

   $$
   \alpha_m = \frac{1}{2} \ln\frac{1 - \epsilon_m}{\epsilon_m}.
   $$

5. **Обновление весов объектов**

   $$
   w_i^{(m+1)} = \frac{w_i^{(m)} \exp\bigl(-\alpha_m y_i h_m(x_i)\bigr)}{Z_m},
   $$

   где нормировочный множитель $Z_m$ обеспечивает $\sum_i w_i^{(m+1)} = 1$.

6. **Финальное решение**

   $$
   F(x) = \operatorname{sign}\bigl(\sum_{m=1}^M \alpha_m h_m(x)\bigr).
   $$

- Тенденция к переобучению при большом числе итераций и слабых шумоустойчивых базовых моделях.
- Чувствительность к выбросам из‑за экспоненциального обновления весов.

<img src="../../images/boosting_classifier.png" alt="Boosting" width="600">

##### Gradient Boosting

**Gradient Boosting** – обобщает идею бустинга как решение задачи градиентного спуска в функциональном пространстве: ансамбль строится итеративно, минимизируя выбранную функцию потерь $L(y, F(x))$.

1. **Инициализация ансамбля**

   $$
   F_0(x) = \arg\min_{\gamma} \sum_{i=1}^N L\bigl(y_i, \gamma\bigr).
   $$

2. **Итерации $m=1 \dots M$**:

   1. **Вычисление псевдо-остатков**

      $$
      r_i^{(m)} = -\left.\frac{\partial L(y_i, F(x_i))}{\partial F(x_i)}\right|_{F=F_{m-1}}.
      $$

      Если используется квадратичная ошибка (MSE): $L = \tfrac12 (y - F)^2$ $\implies r_i^{(m)} = y_i - F_{m-1}(x_i)$.
        
   2. **Обучение базового регрессора** $h_m(x)$ на данных $(x_i, r_i^{(m)})$.
   3. **Поиск шагового коэффициента**

      $$
      \gamma_m = \arg\min_{\gamma} \sum_{i=1}^N L\bigl(y_i, F_{m-1}(x_i) + \gamma\,h_m(x_i)\bigr).
      $$
   4. **Обновление ансамбля**

      $$
      F_m(x) = F_{m-1}(x) + \eta\,\gamma_m\,h_m(x),
      $$

      где $\eta\in (0,1]$ — темп обучения (learning rate).

3. **Финальное предсказание**

   * Для задач регрессии: $\hat y = F_M(x)$.
   * Для задач классификации: $\hat y = \operatorname{sign}\bigl(F_M(x)\bigr)$ или через сигмоиду/softmax для вероятностных оценок.



**Регуляризация и гиперпараметры**

Для предотвращения переобучения и повышения устойчивости применяются:

- **Темп обучения** $\eta$: маленькие значения (например, 0.01–0.1) замедляют обучение.
- **Глубина деревьев**: ограничение максимальной глубины или числа листьев.
- **Количество итераций** $M$: число деревьев.
- **Подвыборка (subsampling)**: случайная выборка $\alpha N$ объектов без замены при обучении каждого дерева.
- **Регуляризация значений листьев**: L1/L2-регуляризация на выходные значения (например, в XGBoost).

**Современные реализации** градиентного бустинга значительно расширяют базовую идею функционального градиентного спуска, предлагая улучшения по скорости обучения, устойчивости к переобучению и работе с категориальными данными.

##### XGBoost (Extreme Gradient Boosting)

XGBoost – одна из наиболее популярных реализаций градиентного бустинга, отличающаяся высокой скоростью и качеством. Основные особенности:

* **Второй порядок оптимизации**: в отличие от классического бустинга, XGBoost использует не только градиенты, но и гессианы (вторые производные функции потерь), что позволяет более точно выбирать шаги обновления.
* **Регуляризация**: в функцию потерь добавляются L1- и L2-регуляризаторы, что повышает устойчивость к переобучению:

  $$
  \mathcal{L}^{(t)} = \sum_{i=1}^N L(y_i, \hat{y}_i^{(t)}) + \sum_{k=1}^{t} \Omega(h_k), \quad \Omega(h) = \gamma T + \frac{1}{2}\lambda \sum_{j=1}^{T} w_j^2
  $$

  где $T$ — число листьев, $w_j$ — вес узла.
* **Пропущенные значения**: встроенные механизмы для автоматического определения направления разветвления при отсутствующих значениях.
* **Параллелизм**: обучение и построение гистограмм для разбиения признаков выполняется параллельно.

<div class="alert alert-info">
XGBoost строит деревья по принципу: «Строим дерево последовательно по уровням до достижения максимальной глубины». Отдельного ограничения на количество вершин нет, так как оно естественным образом получается из ограничения на глубину дерева. В XGBoost деревья «стремятся» быть симметричными по глубине, и в идеале получается полное бинарное дерево, если это не противоречит другим ограничениям (например, ограничению на минимальное количество объектов в листе). Такие деревья обычно являются более устойчивыми к переобучению.
</div>

<img src="https://yastatic.net/s3/education-portal/media/tree_xgboost_e0caf19935_5029e855a6.webp" alt="Описание изображения" width="600">

##### LightGBM

LightGBM – библиотека от Microsoft, ориентированная на масштабируемость и скорость при сохранении качества модели:

* **Leaf-wise рост деревьев**: вместо уровня (level-wise) выбирается лист с наибольшим вкладом в уменьшение функции потерь, что ускоряет сходимость, но может привести к переобучению.
* **Гистограммное разбиение**: признаки бинируются (дискретизируются) в фиксированное число корзин (bins), что сокращает потребление памяти и ускоряет поиск разбиений.
* **Градиентное сэмплирование (GOSS)**: важные (большие) по градиенту примеры используются всегда, остальные — с вероятностью, что сохраняет информативность и ускоряет обучение.
* **Экстремально эффективен на больших и разреженных данных**.

<div class="alert alert-info">
LightGBM строит деревья по принципу: «На каждом шаге делим вершину с наилучшим скором», а основным критерием остановки выступает максимально допустимое количество вершин в дереве. Это приводит к тому, что деревья получаются несимметричными, то есть поддеревья могут иметь разную глубину – например, левое поддерево может иметь глубину 2, а правое может разрастись до глубины 15.
</div>

<img src="https://yastatic.net/s3/education-portal/media/tree_lightgbm_b27000abd5_dcdf18005a.webp" alt="Описание изображения" width="600">

##### CatBoost

CatBoost – библиотека от Яндекса, разработанная специально для эффективной обработки категориальных признаков:

* **Порядковый бустинг** (ordered boosting): устранение таргет-шифта за счёт особой последовательности построения модели, при которой каждый базовый алгоритм обучается только на части данных, не содержащей целевой переменной для текущего объекта.
* **Обработка категориальных признаков**: автоматически применяет методы target encoding и счётчиков с регуляризацией, устраняя необходимость в предварительной подготовке.
* **Симметричные деревья**: все разветвления принимаются одновременно на всех уровнях, что ускоряет предсказание и делает модель более устойчивой.
* **Поддержка мультиклассовой классификации, нерегулярных задач и использования CPU/GPU.**

<div class="alert alert-info">
CatBoost строит деревья по принципу: «Все вершины одного уровня имеют одинаковый предикат». Одинаковые сплиты во всех вершинах одного уровня позволяют избавиться от ветвлений (конструкций if-else) в коде инференса модели с помощью битовых операций и получить более эффективный код, который в разы ускоряет применение модели, в особенности в случае применения на батчах.
</div>

Кроме этого, такое ограничение на форму дерева выступает в качестве сильной регуляризации, что делает модель более устойчивой к переобучению. Основной критерий остановки, как и в случае XGBoost, – ограничение на глубину дерева. Однако, в отличие от XGBoost, в CatBoost всегда создаются полные бинарные деревья, несмотря на то, что в некоторые поддеревья может не попасть ни одного объекта из обучающей выборки.

<img src="https://yastatic.net/s3/education-portal/media/tree_catboost_d501cbfe52_d44e880b4e.webp" alt="Описание изображения" width="600">

In [65]:
!pip install xgboost -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

In [ ]:
np.random.seed(42)
n_train = 20
noise_std = 0.3
n_experiments = 50
x_grid = np.linspace(0, 2 * np.pi, 100).reshape(-1, 1)
y_true = true_function(x_grid.ravel())

In [ ]:
tree_model = DecisionTreeClassifier(random_state=42, max_depth=5)
tree_predictions = run_experiments(tree_model, n_train, 
                                   noise_std, x_grid, n_experiments)
tree_bias, tree_variance = bias_variance_decomposition(tree_predictions, y_true)

xgb_model = XGBClassifier(n_estimators=50,
                          max_depth=10,
                          subsample=0.8,
                          random_state=42)
xgb_predictions = run_experiments(xgb_model, n_train, 
                                  noise_std, x_grid, n_experiments)
xgb_bias, xgb_variance = bias_variance_decomposition(xgb_predictions, y_true)

model_names = ["Decision Tree", "Boosting"]
bias_values = [tree_bias, xgb_bias]
variance_values = [tree_variance, xgb_variance]

display_results(model_names, bias_values, variance_values)

### Stacking

Метод стекинга объединяет прогнозы нескольких базовых моделей посредством мета-модели. Базовые модели (уровень-0) выдают предсказания, которые затем используются в качестве входных признаков для мета-модели (уровень-1). Итоговая модель может быть записана как:
  $$
  f_{\text{stack}}(x) = g\Bigl( h_1(x), h_2(x), \dots, h_M(x) \Bigr),
  $$
  где $ g $ – обучаемая мета-функция.

<div class="alert alert-info">
Позволяет объединить сильные стороны различных моделей, что зачастую приводит к улучшению качества предсказаний.
</div>

<img src="../../images/stacking_model.jpg" alt="Stacking" width="600">

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier

In [ ]:
np.random.seed(42)
n_train = 20
noise_std = 0.3
n_experiments = 50
x_grid = np.linspace(0, 2 * np.pi, 100).reshape(-1, 1)
y_true = true_function(x_grid.ravel())

In [ ]:
tree_model = DecisionTreeClassifier(random_state=42, max_depth=5)
tree_predictions = run_experiments(tree_model, n_train, noise_std, x_grid, n_experiments)
tree_bias, tree_variance = bias_variance_decomposition(tree_predictions, y_true)

estimators = [
    ('dt', DecisionTreeClassifier(random_state=42)),
    ('lr', LogisticRegression())
]

stack_model = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(),
    cv=5,
    n_jobs=-1
)

stack_predictions = run_experiments(stack_model, n_train, noise_std, x_grid, n_experiments)
stack_bias, stack_variance = bias_variance_decomposition(stack_predictions, y_true)

model_names = ["Decision Tree", "Stacking"]
bias_values = [tree_bias, stack_bias]
variance_values = [tree_variance, stack_variance]

display_results(model_names, bias_values, variance_values)

### Blending

Блэндинг – это ансамблевый приём, который заключается в следующем:
- обучаем несколько базовых моделей на тренировочных данных
- выделяем небольшую часть данных (holdout) и получаем на ней предсказания базовых моделей
- на этих предсказаниях обучаем **мета-модель** (обычно простая логистическая регрессия или линейная регрессия).

На тесте комбинируем предсказания базовых моделей через обученную мета-модель.

По задумке, мета-модель должна научится комбинировать слабые и разные по поведению базовые модели, но для неё используются **предсказания на holdout наборе**, а не out-of-fold (как при стэкинге). Из-за этого метод менее стабилен на небольших датасетах, ввиду необходимости иметь holdout набор.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [ ]:
X, y = load_breast_cancer(return_X_y=True)

X_rest, X_test, y_rest, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

X_train, X_blend, y_train, y_blend = train_test_split(X_rest, y_rest, test_size=0.25, random_state=42, stratify=y_rest)

In [ ]:
base_models = [
    ("rf", RandomForestClassifier(n_estimators=200, random_state=1)),
    ("gb", GradientBoostingClassifier(n_estimators=200, random_state=2))
]

for name, model in base_models:
    model.fit(X_train, y_train)

In [ ]:
blend_preds = []
for name, model in base_models:
    p = model.predict_proba(X_blend)[:, 1]
    blend_preds.append(p.reshape(-1, 1))
X_blend_meta = np.hstack(blend_preds)

meta = LogisticRegression()
meta.fit(X_blend_meta, y_blend)

In [ ]:
test_preds = []
for name, model in base_models:
    p = model.predict_proba(X_test)[:, 1]
    test_preds.append(p.reshape(-1, 1))
X_test_meta = np.hstack(test_preds)
y_pred_meta = meta.predict_proba(X_test_meta)[:, 1]

print("ROC AUC (meta):", roc_auc_score(y_test, y_pred_meta))

In [ ]:
for name, model in base_models:
    p = model.predict_proba(X_test)[:, 1]
    print(f"ROC AUC ({name}):", roc_auc_score(y_test, p))

## Ансамблевые методы для задачи регрессии

Ансамблевые методы могут используются и в задачах регрессии. Однако, итоговая аппроксимация исходной зависимости имеет характер разбиения на регионы, внутри которых модель ведёт себя как некоторая простая функция – обычно постоянная или, при специальных модификациях, линейная.

Классические регрессионные деревья (например, `DecisionTreeRegressor`) разделяют пространство признаков на непересекающиеся регионы $ R_1, R_2, \dots, R_n $. В каждом регионе дерево присваивает константное значение, как правило, равное среднему значению целевой переменной для объектов, попавших в этот регион:
$$
f(x) = \sum_{i=1}^{n} c_i \cdot \mathbb{I}(x \in R_i)
$$
где:
- $ c_i $ – среднее значение отклика для $ R_i $,
- $\mathbb{I}(x \in R_i)$ – индикатор, равный 1, если $x$ принадлежит $R_i$.

Таким образом, **если базовой моделью является дерево решений с константным прогнозом в листьях**, восстановленная функция будет иметь вид кусочно постоянного приближения. Усредняя, можно искусственно сгладить эти переходы, но принцип регионального разбиения пространства сохраняется.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [ ]:
def true_function(x):
    """
    Истинная функция: sin(x)
    """
    return np.sin(x)

In [ ]:
np.random.seed(42)
n_samples = 10
X = np.random.uniform(0, 2*np.pi, n_samples)
X = np.sort(X)
y_true = true_function(X)
noise = np.random.normal(0, 0.2, n_samples)
y = y_true # + noise

X = X.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=42)

In [ ]:
rf_model = RandomForestRegressor(n_estimators=20, random_state=42)
gb_model = XGBRegressor(n_estimators=20,
                        random_state=42,
                        objective="reg:squarederror",
                        verbosity=0)

rf_model.fit(X_train, y_train)
gb_model.fit(X_train, y_train)

In [ ]:
y_test_pred_rf = rf_model.predict(X_test)
y_test_pred_gb = gb_model.predict(X_test)
mse_rf = mean_squared_error(y_test, y_test_pred_rf)
mse_gb = mean_squared_error(y_test, y_test_pred_gb)
print(f"MSE (Random Forest Regressor): {mse_rf:.2f}")
print(f"MSE (XGBoost Regressor): {mse_gb:.2f}")

In [ ]:
x_grid = np.linspace(0, 2*np.pi, 500).reshape(-1, 1)
y_grid_true = true_function(x_grid.ravel())
y_grid_rf = rf_model.predict(x_grid)
y_grid_gb = gb_model.predict(x_grid)

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(x_grid, y_grid_true, color='black', lw=2, label='Grand Truth')
plt.plot(x_grid, y_grid_rf, lw=2, label=f'Random Forest')
plt.plot(x_grid, y_grid_gb, lw=2, label=f'XGBoost')
plt.scatter(X, y, color='gray', alpha=0.6, label='Data')
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

<div class="alert alert-info">
Вы можете попрактиковаться на конкретных данных, выполнив задания из
<a href="../../Workshops/week03_boosting.ipynb" target="_blank">ноутбука</a>
</div>